In [1]:
pip install requests pillow

Note: you may need to restart the kernel to use updated packages.


In [3]:
import tkinter as tk
from tkinter import messagebox
import requests
from PIL import Image, ImageTk
from io import BytesIO

# Replace with your actual API key
API_KEY = "  "
BASE_URL = "https://api.openweathermap.org/data/2.5/weather"

def get_weather():
    city = city_entry.get()
    if not city:
        messagebox.showwarning("Input Error", "Please enter a city name.")
        return
    
    params = {
        "q": city,
        "appid": API_KEY,
        "units": "metric"
    }

    try:
        response = requests.get(BASE_URL, params=params)
        data = response.json()

        if data["cod"] == 401:
            messagebox.showerror("Invalid API Key", "Your API key is invalid. Please replace it with a valid key.")
            return
        elif data["cod"] != 200:
            messagebox.showerror("Error", f"City not found: {city}")
            return


        temp = data["main"]["temp"]
        humidity = data["main"]["humidity"]
        desc = data["weather"][0]["description"]
        icon_code = data["weather"][0]["icon"]
        icon_url = f"http://openweathermap.org/img/wn/{icon_code}@2x.png"

        # Display weather info
        result_label.config(
            text=f"Temperature: {temp}°C\nHumidity: {humidity}%\nDescription: {desc.title()}"
        )

        # Display icon
        icon_response = requests.get(icon_url)
        icon_image = Image.open(BytesIO(icon_response.content))
        icon_photo = ImageTk.PhotoImage(icon_image)
        icon_label.config(image=icon_photo)
        icon_label.image = icon_photo  # Keep reference

    except requests.exceptions.RequestException as e:
        messagebox.showerror("Error", f"Failed to retrieve data: {e}")

# GUI setup
app = tk.Tk()
app.title("Weather App")
app.geometry("350x400")
app.resizable(False, False)

tk.Label(app, text="Enter City Name:", font=("Helvetica", 12)).pack(pady=10)

city_entry = tk.Entry(app, font=("Helvetica", 14), justify="center")
city_entry.pack(pady=5)

tk.Button(app, text="Get Weather", command=get_weather, font=("Helvetica", 12)).pack(pady=10)

icon_label = tk.Label(app)
icon_label.pack(pady=10)

result_label = tk.Label(app, text="", font=("Helvetica", 12), justify="center")
result_label.pack(pady=10)

app.mainloop()
